## CRS

**Observations**  
-Taux de remplissage  
-Distribution des datasets par EPSG


In [1]:
import pandas as pd
from utils import wrapper_engine, get_value_from_json_serialization, head_as_table

engine = wrapper_engine('config.ini')

# Load data
with engine.connect() as connection:    
    package_extra = pd.read_sql_table(
        table_name="package_extra", 
        con=connection,
        schema="public",
        )
    
package_extra = package_extra[package_extra.key.isin(["crs"])].sort_values(by=["package_id", "key"])


**Nombre de datasets avec la métadonnée CRS**

In [2]:
head_as_table(package_extra[["package_id"]].describe().reset_index(), n_rows=5)

**tableau complet :** 4 lignes × 2 colonnes

index,package_id
count,21487
unique,21487
top,00007ee6-8353-4f07-bd0c-e5ee0fabb0cb
freq,1


In [12]:
package_extra = pd.pivot_table(package_extra, index=["package_id"], values=["value"], columns=["key"], aggfunc='first')
package_extra = package_extra.droplevel(0, axis=1).reset_index().rename_axis(columns=None)

# EPSG value - alternative approach to the split from the URI: reading the EPSG from the XML file (see utils.get_value_from_xml)
# When multiple EPSG values are available (seen only once), take first option
package_extra["EPSG"] = package_extra["crs"].apply(lambda x: get_value_from_json_serialization(x, "uri")).apply(lambda x:  x[0].split('/')[-1] if len(x) > 0 else 'Unknown')

In [13]:
package_extra[["EPSG", "package_id"]].rename(columns={"package_id":"Datasets [count]"}).groupby(["EPSG"], as_index=False).count().sort_values(by=["Datasets [count]"], ascending=False).transpose().style.hide(axis="columns")

EPSG,2154,4171,4326,27572,3857,4463,27561,27573,2972,3947,3948,3950
Datasets [count],21243,196,28,7,3,3,2,1,1,1,1,1




|EPSG | Name |Area of use | Unit | Accuracy |
|:---:|---|---|:---:|:---:|
|[2154](https://epsg.io/2154) | RGF93 v1 / Lambert-93 -- France | France - onshore and offshore, mainland and Corsica (France métropolitaine including Corsica). | metre | 1.0m |
|[27561](https://epsg.io/27561) | NTF (Paris) / Lambert Nord France | France mainland onshore north of 53.5 grads North (48°09'N). | metre | 2.0m |
|[27572](https://epsg.io/27572) | NTF (Paris) / Lambert zone II | France mainland onshore between 50.5 grads and 53.5 grads North (45°27'N to 48°09'N). Also used over all onshore mainland France. | metre | 2.0m |
|[27573](https://epsg.io/27573) | NTF (Paris) / Lambert zone III | France - mainland onshore south of 50.5 grads North (45°27'N). | metre | 2.0m |
|[2972](https://epsg.io/2972) | RGFG95 / UTM zone 22N | French Guiana - east of 54°W, onshore and offshore. | metre | 1.0m|
|[3857](https://epsg.io/3857) | WGS 84 / Pseudo-Mercator -- Spherical Mercator, Google Maps, OpenStreetMap, Bing, ArcGIS, ESRI | World between 85.06°S and 85.06°N. | metre | up to 43km |
|[3947](https://epsg.io/3947) | RGF93 v1 / CC47 | France - mainland onshore between 46°N and 48°N. | metre | 1.0m |
|[3948](https://epsg.io/3948)  | RGF93 v1 / CC48 | France - mainland onshore between 47°N and 49°N. | metre | 1.0m |
|[3950](https://epsg.io/3950) |RGF93 v1 / CC50 | France - mainland onshore north of 49°N. | metre | 1.0m |
|[4171](https://epsg.io/4171) |RGF93 v1 | France - onshore and offshore, mainland and Corsica (France métropolitaine including Corsica). | degree | - |
|[4326](https://epsg.io/4326) | WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS | World | degree | - |
|[4463](https://epsg.io/4463) | RGSPM06 | St Pierre and Miquelon - onshore and offshore. | degree | - |